### Create autoencoder model

In [ ]:
%%bigquery
CREATE MODEL `gurkomal-playground.avoxi_workshop.autoencoder_v1`
OPTIONS(
  model_type='AUTOENCODER',
  batch_size=32,
  hidden_units=[8, 4, 2, 4, 8],
  activation_fn='RELU',
  optimizer='ADAM',
  learn_rate=0.01,
  max_iterations=10,
  early_stop=TRUE
) AS (
    SELECT
        packet_loss,
        duration,
        jitter,
        mean_opinion_score
    FROM
        `gurkomal-playground.avoxi_workshop.anomaly_labelled_data`
);

### Hyperparameter tuning autoencoder

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL `avoxi_workshop.autoencoder_v1`
    OPTIONS(
        MODEL_TYPE='AUTOENCODER',
        BATCH_SIZE=32,
        HIDDEN_UNITS=HPARAM_CANDIDATES([STRUCT([8, 4, 2, 4, 8])]),
        HPARAM_TUNING_OBJECTIVES=['MEAN_SQUARED_ERROR'],
        HPARAM_TUNING_ALGORITHM='VIZIER_DEFAULT',
        NUM_TRIALS=4,
        ACTIVATION_FN = HPARAM_CANDIDATES(['RELU', 'SIGMOID']),
        OPTIMIZER = HPARAM_CANDIDATES(['ADAM', 'SGD'])
) AS (
 SELECT
   packet_loss,
   duration,
   jitter,
   mean_opinion_score
 FROM
   `gurkomal-playground.avoxi_workshop.anomaly_labelled_data_v2`
);

### Inspect reconstruction loss of model

In [ ]:
%%bigquery
SELECT *
FROM ML.RECONSTRUCTION_LOSS(
MODEL `avoxi_workshop.autoencoder_v1`,
    (SELECT
     duration,
     jitter,
     packet_loss,
     mean_opinion_score,
     is_anomaly
   FROM `gurkomal-playground.avoxi_workshop.anomaly_labelled_data_v2`),
   STRUCT(
     4 AS trial_id)
 )
 WHERE is_anomaly = FALSE AND mean_squared_log_error < 1

### Review all trial results from hyperparameter tuning job

In [ ]:
%%bigquery
SELECT
   *
FROM
   ML.TRIAL_INFO(MODEL `avoxi_workshop.autoencoder_v1`)

### Detect Anomalies using autoencoder and store as results table

In [ ]:
%%bigquery
CREATE TABLE avoxi_workshop.results AS(
SELECT
  *
FROM
  ML.DETECT_ANOMALIES(MODEL `avoxi_workshop.autoencoder_v1`,
    STRUCT(0.04 AS contamination),
    TABLE `gurkomal-playground.avoxi_workshop.anomaly_labelled_data_v3`))

### Validate anomalies - predicted vs actual results

In [ ]:
%%%bigquery
SELECT
  p.is_anomaly as predicted_result,
  a.is_anomaly as actual_result,
  p.`164_from_caller_id`,
  p.`164_to_caller_id`
FROM `gurkomal-playground.avoxi_workshop.anomaly_labelled_data_v2` a
LEFT JOIN `gurkomal-playground.avoxi_workshop.results` p
ON
  p.`164_from_caller_id` = a.`164_from_caller_id` 
  AND p.`164_to_caller_id`= a.`164_to_caller_id`
WHERE
  p.is_anomaly= a.is_anomaly

### Gemini 1.5 Pro Text generation
* Using only the table values & schema, generate potential anomalies found (Country and Caller ID)

In [ ]:
%%bigquery
WITH TableA_Aggregated AS (
  SELECT STRING_AGG(
    concat(CAST(trial_id as STRING),',',CAST(is_anomaly as STRING),',',CAST(mean_squared_error as STRING),',',CAST(start_time as STRING),',',CAST(`164_from_caller_id` as STRING),',',CAST(`164_to_caller_id` as STRING),',',CAST(data_center as STRING),',',CAST(duration as STRING),',',CAST(packet_loss as STRING),',',CAST(mean_opinion_score as STRING),',',CAST(jitter as STRING),',',CAST(day as STRING),',',CAST(hour as STRING),',',CAST(from_country as STRING),',',CAST(to_country as STRING))
    ) AS all_rows
  FROM `gurkomal-playground.avoxi_workshop.results` t
),
result_schema AS (
  SELECT
  STRING_AGG(
    FORMAT("%s %s", column_name, data_type), 
    ", " 
    ORDER BY ordinal_position
  ) AS schema_string
  FROM
    `gurkomal-playground.avoxi_workshop`.INFORMATION_SCHEMA.COLUMNS
  WHERE
    table_name = "results"
)
SELECT *
FROM
  ML.GENERATE_TEXT(
    MODEL `gurkomal-playground.avoxi_workshop.gemini_pro`,
    (
      SELECT CONCAT('Given the following table output and schema, summarize which countries and caller IDs were classified as anomalies\n', all_rows,schema_string) AS PROMPT
      FROM TableA_Aggregated, result_schema
    ),
    STRUCT(
      0.8 AS temperature,
      1024 AS max_output_tokens,
      0.95 AS top_p,
      40 AS top_k
    )
  )

In [ ]:
# %%bigquery
# SELECT
#    *
# FROM
#    ML.PREDICT(MODEL `avoxi_workshop.autoencoder_v1`,
#      TABLE `gurkomal-playground.avoxi_workshop.anomaly_labelled_data_v2`
# )